In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import argparse
import os
import glob
import sys
from pathlib import Path
import time
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [ ]:
# Dataset directories
BASE = "/kaggle/input/MABe-mouse-behavior-detection"  # adjust to your dataset
train_tracking= BASE / "train_tracking"
train_annotation = BASE / "train_annotation"
test_tracking = BASE / "test_tracking"

In [ ]:
# Working directory for temporary outputs
WORK_DIR = Path("/kaggle/working")
WORK_DIR.mkdir(parents=True, exist_ok=True)

# Load main CSV mappings
train_map = pd.read_csv(DATA_DIR / "train.csv")
test_map = pd.read_csv(DATA_DIR / "test.csv")

In [ ]:
# Pick feature columns automatically from first tracking file
example_tracking_file = next(train_tracking.iterdir())
tracking_example = pd.read_csv(example_tracking_file)
feature_columns = [col for col in tracking_example.columns if col not in ['video_frame','agent_mouse_id','target_mouse_id']]

# Pick label column dynamically from first annotation file
example_annotation_file = next(train_annotation.iterdir())
annotation_example = pd.read_csv(example_annotation_file)
label_columns = [col for col in annotation_example.columns if col not in ['video_frame','agent_mouse_id','target_mouse_id']]

In [ ]:
def merge_tracking_annotation(tracking_file, annotation_file):
    tracking_df = pd.read_csv(tracking_file)
    annotation_df = pd.read_csv(annotation_file)

    # Add action column
    tracking_df['action'] = 'none'

    # Map annotation to frames
    for _, row in annotation_df.iterrows():
        agent = row['agent_id']
        start = row['start_frame']
        stop = row['stop_frame']
        action = row['action']

        mask = (tracking_df['mouse_id'] == agent) & \
               (tracking_df['video_frame'] >= start) & \
               (tracking_df['video_frame'] <= stop)
        tracking_df.loc[mask, 'action'] = action

    return tracking_df
